In [7]:
from ela_feature_definition import ela_feature_names
import pandas as pd
import random
import numpy as np
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.cluster import *
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import *
from sklearn.metrics.cluster import adjusted_rand_score
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_samples, silhouette_score
import numpy as np

In [8]:
save_clusters=False

In [9]:
def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [10]:
def evaluate_clustering(df, cluster_count):
    results_for_cluster={}
    labels =  AgglomerativeClustering(n_clusters=cluster_count, affinity='cosine', linkage='complete').fit(df).labels_
    results_for_cluster['labels'] = labels
    samples_per_cluster={k[0]:v for k,v in pd.DataFrame(labels).value_counts().to_dict().items()}
    results_for_cluster['samples_per_cluster']=samples_per_cluster
    results_for_cluster['silhouette_score'] =silhouette_score(df, labels, metric='cosine')
    sample_silhouette_values = silhouette_samples(df, labels, metric='cosine')
    return results_for_cluster

In [11]:
ela = pd.read_csv('data/aggregated_ela_representation_dropna.csv',index_col=[0,1,2])
ela=ela[set(ela_feature_names).intersection(ela.columns)]

In [12]:
clustering_results={}
for cluster_count in range(2, 100):
    clustering_results[f'cluster_{cluster_count}']=evaluate_clustering(ela.copy(),cluster_count)
clustering_results_df=pd.DataFrame(clustering_results).T

In [13]:
clustering_results_df

,labels,samples_per_cluster,silhouette_score
cluster_2,"[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 205, 0: 16}",0.962152
cluster_3,"[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: 205, 1: 14, 2: 2}",0.933072
cluster_4,"[3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","{3: 199, 0: 14, 1: 6, 2: 2}",0.907603
cluster_5,"[3, 3, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","{3: 199, 2: 12, 1: 6, 0: 2, 4: 2}",0.905568
cluster_6,"[1, 1, 2, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 199, 2: 12, 0: 6, 4: 2, 3: 1, 5: 1}",0.902641
...,...,...,...
cluster_95,"[52, 8, 49, 47, 51, 40, 76, 40, 17, 40, 53, 61...","{16: 52, 10: 10, 17: 9, 12: 8, 1: 6, 4: 6, 11:...",0.45906
cluster_96,"[52, 8, 49, 95, 51, 40, 76, 40, 17, 40, 53, 61...","{16: 52, 10: 10, 17: 9, 12: 8, 4: 6, 1: 6, 5: ...",0.452432
cluster_97,"[52, 8, 49, 95, 51, 40, 76, 40, 17, 40, 53, 61...","{7: 52, 10: 10, 17: 9, 12: 8, 0: 6, 4: 6, 5: 5...",0.446204
cluster_98,"[52, 8, 49, 95, 51, 40, 76, 40, 17, 40, 53, 61...","{7: 52, 10: 10, 17: 9, 12: 8, 4: 6, 5: 5, 6: 5...",0.442648


In [14]:
selected_cluster_counts = [12]
metric='cosine'
for cluster_count in selected_cluster_counts:
    agglomerative = AgglomerativeClustering(n_clusters=cluster_count,affinity =metric ,linkage='complete')
    labels=agglomerative.fit(ela).labels_
    ela_copy=ela.copy()
    ela_copy[f'clustering_{cluster_count}_clusters']=labels
    if save_clusters:
        ela_copy.to_csv(f'data/hierarchical_clustering_{cluster_count}.csv')

SUBCLUSTERING

In [15]:
cluster_12_df = pd.read_csv('data/hierarchical_clustering_12.csv',index_col=[0,1,2])
cluster_label_column='clustering_12_clusters'
cluster_sizes=pd.DataFrame(cluster_12_df[cluster_label_column]).value_counts()
max_cluster_size=cluster_sizes.max()
largest_cluster_id = cluster_sizes[cluster_sizes==max_cluster_size].index[0][0]
print(largest_cluster_id)
ela=cluster_12_df[cluster_12_df[cluster_label_column]==largest_cluster_id]
ela=ela[set(ela_feature_names).intersection(ela.columns)]
removed_instances=cluster_12_df[cluster_12_df[cluster_label_column]!=largest_cluster_id]
initial_cluster_count=cluster_sizes.shape[0]

print(initial_cluster_count)

0
12


In [16]:
clustering_results={}
df=ela
for cluster_count in range(2, 60):
    clustering_results[f'cluster_{cluster_count}']=evaluate_clustering(df,cluster_count)
clustering_results_df=pd.DataFrame(clustering_results).T

In [17]:
selected_cluster_counts = [10,15]

metric='cosine'
for cluster_count in selected_cluster_counts:
    agglomerative = AgglomerativeClustering(n_clusters=cluster_count,affinity =metric ,linkage='complete')
    labels=agglomerative.fit(ela).labels_
    elements_per_cluster = pd.DataFrame(labels).value_counts()
    ela[f'subclustering_{cluster_count}_clusters']=[initial_cluster_count + l for l in labels]
    removed_instances[f'subclustering_{cluster_count}_clusters']=removed_instances[cluster_label_column]
    
all_instances=ela.append(removed_instances)
if save_clusters:
    all_instances.to_csv('data/subclustering.csv')

/tmp/ipykernel_20924/686659388.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  removed_instances[f'subclustering_{cluster_count}_clusters']=removed_instances[cluster_label_column]
/tmp/ipykernel_20924/686659388.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  removed_instances[f'subclustering_{cluster_count}_clusters']=removed_instances[cluster_label_column]
